In [ ]:
import glob
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import joblib

In [ ]:
path_df = r"Table" + "\\"
path_model = r"Model" + "\\"
path_fig = r"Figure" + "\\"
path_temprature = r"Data\10_day_mean" + "\\"

N1 = 50  # Number of initial features
N2 = 20  # Number of final features

In [ ]:
df = []
file_list = glob.glob(path_temprature + "*.csv")
file_list.sort()

for file in file_list:
    df1 = pd.read_csv(file)
    df1.dropna(subset=["Temperature"], inplace=True)
    df.append(df1)

df = pd.concat(df, axis=0, ignore_index=True)
features = df.columns.drop(["River", "Station", "Temperature", "Date"])
features = [f for f in features if "basin_" not in f]

X_train, X_test, y_train, y_test = train_test_split(
    df[features], df["Temperature"], test_size=0.2, random_state=42
)

# Save the train data
X_train.to_csv(path_df + "X_train.csv", index=False)
y_train.to_csv(path_df + "y_train.csv", index=False)

# XGBoost

## Feature Selection

In [ ]:
xgb = XGBRegressor(n_estimators=50, random_state=42, n_jobs=-1)
rfe = RFE(estimator=xgb, n_features_to_select=N2, step=1)

rfe.fit(X_train, y_train)
ranking = rfe.ranking_
ranking = pd.DataFrame(ranking, index=X_train.columns, columns=["Rank"]).sort_values(
    by="Rank", ascending=True
)
ranking.index.name = "Feature"
features_selected = ranking[ranking["Rank"] == 1]
features_selected.to_csv(path_df + "Selected_features_XGBoost.csv")
feature_names = features_selected.index

X_train2 = X_train[feature_names]
X_test2 = X_test[feature_names]

## Hyperparameter Tuning

In [ ]:
# Define the hyperparameter grid for XGBoost
param_grid = {
    "max_depth": [3, 4],
    "learning_rate": [0.005, 0.01, 0.015],
    "n_estimators": [300, 400],
    "subsample": [0.6, 0.7, 0.8],
    "colsample_bytree": [0.5, 0.7, 1],
}

# Search for the best hyperparameters using RandomizedSearchCV
xgb = XGBRegressor(random_state=42, n_jobs=-1)
xgb_random = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    n_iter=100,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1,
)
xgb_random.fit(X_train2, y_train)

# Get the best model and its hyperparameters
model = xgb_random.best_estimator_
best_params = xgb_random.best_params_

# Save the best parameters as a JSON file
with open(path_model + "XGBoost_best_params.json", "w") as f:
    json.dump(best_params, f)

In [ ]:
X = df[feature_names]
y = df["Temperature"]
model.fit(X, y)

file_model = path_model + f"XGBoost_model.joblib"
joblib.dump(model, file_model)